In [104]:
token= "d24465a1e125f57c83e39885491e73f9e4487de8faba4e82bf1288fc631543b4b691b7226d9d9b40e3a3e"
myid = "6045249"

# Подгружаем уже знакомую нам библиотеку для запросов
import requests

def Vk_downloader(method,parameters,token = token):
    url = 'https://api.vk.com/method/'+method+'?'+parameters+'&access_token='+str(token)
    response = requests.get(url) 
    return(response.json())

In [105]:
# Проверяем всё ли работает на своём имени.
Vk_downloader('users.get','user_ids=6045249')

{'response': [{'first_name': 'Филипп',
   'last_name': 'Ульянкин',
   'uid': 6045249}]}

Перед нами стоит довольно мсштабная и сложная задача. Мы должны написать код, который будет делать две вещи:

* Код должен брать какого-то конкретного человека и находить все лоисы, которые он ставил в последнее время.
* Код должен находить все лоисы, которые были поставлены этому человеку.

Для простоты будем руководствоваться следующими предпосылками. Если он ставит много лайков какой-то тёлке, значит он хочет с ней переспать. Если он ставит много лайков какой-то тёлке, а какая-то тёлка ставит много лайков ему, значит они уже давно спят. Чтож! Включаем свой измухислоноделатель и вперёд! Искать измены и изменщиков. Всё, что однажды попало в интернет остаётся в интернете навечно. Точно также навечно он останется в могиле, когда мы с тобой всё узнаем...


## 1. Его лайкатели

Соберём все лайки, которые есть у нег на странице. Будем собирать лайки со стены и с фоточек.  Этого должно хватить. (ДЛЯ ЧЕГО БЛИН ХВАТИТЬ?!)


In [112]:
his_id = "87385777"

Собираем id всех людей, которые лайкнули его стену и время, в которое они это сделали. Будем собирать информацию годовой свежести. Сначала собираем стену. С неё нам потребуются id записи и время, когда она была размещена.  

In [113]:
# Число записей на стене:
n = Vk_downloader('wall.get','owner_id='+his_id)['response'][0]
n

51

In [114]:
n // 100

0

In [116]:
chto_on_postit =  [ ]
for i in range(0,100*(n//100+1),100):
    wall = Vk_downloader('wall.get','owner_id='+his_id+'&count=100&offset='+str(i))['response']
    chto_on_postit.extend([[item['id'],item['date']] for item in wall[1:]])

len(chto_on_postit)

51

Стена у нас. Теперь возьмём кажду запись и посмотрим на id тех, кто лайкал её.

In [120]:
Vk_downloader('likes.getList','type=post&owner_id='+his_id+'&item_id='+str(chto_on_postit[1][0]))

{'response': {'count': 13,
  'users': [1119778,
   11983762,
   12169511,
   12332013,
   15598144,
   31645230,
   33063426,
   53874035,
   56141079,
   195518047,
   346502968,
   356502688,
   393881886]}}

Собираем информацию о всех записях со стены.

In [121]:
wall_likes = [ ]
for item in chto_on_postit:
    a = Vk_downloader('likes.getList','type=post&owner_id='+his_id+'&item_id='+str(item[0]))
    wall_likes.append([item[1],a['response']['users']])
    
wall_likes[-3:]

[[1351158707,
  [324191161,
   3725990,
   4910969,
   8704213,
   8987491,
   15598144,
   18319207,
   23163529,
   25386440,
   31645230,
   36076632,
   45062739,
   50326247,
   53874035,
   54132581,
   56141079,
   71554513,
   75268658,
   90693963,
   96155503,
   98271934,
   98732311,
   112104602,
   153737703,
   175275930,
   178107745,
   180112558]],
 [1350764396, [31645230, 87385777, 142031968]],
 [1345129215,
  [21345871,
   22765047,
   40267069,
   41415965,
   52726745,
   53874035,
   63813664,
   87385777,
   110391696]]]

Обратим свой взор на собранное нами время постов. В этот вектор попали какие-то странные цифры. Эти цифры означают количество секунд, которое прошло с 1970 года. При парсинге вам часто предстоит сталкиваться с таким форматом записи времени.

In [130]:
wall_likes[0][0]

1431361204

Модуль `datetime` позволяет работать с датами и временем. В том числе он позволяет перевести секунды в нормальный формат.

In [127]:
import datetime
datetime.datetime.fromtimestamp(1472484553).strftime('%Y-%m-%d %H:%M:%S')

'2016-08-29 18:29:13'

In [134]:
datetime.datetime.fromtimestamp(1472484553).strftime('%Y-%m-%d')

'2016-08-29'

In [139]:
datetime.datetime.fromtimestamp(1472484553).strftime('%m-%d')

'08-29'

Можно обрубить время до того состояния, которое нам нужно и перевести в текстовый формат. Также можно скоадывать и вычитать даты. Например.

In [148]:
dt1 = datetime.datetime.fromtimestamp(1472484553)
dt2 = datetime.datetime.fromtimestamp(1172484553)
print(dt1.strftime('%Y-%m-%d %H:%M:%S'))
print(dt2.strftime('%Y-%m-%d %H:%M:%S'))
dt3 = dt1 - dt2
dt3.days

2016-08-29 18:29:13
2007-02-26 13:09:13


3472

Собираем id всех людей, которые лайкнули его фотки и время, когда фото было опубликовано.

In [103]:
# Все айдишники альбомов: 
albums = [item['aid'] for item in Vk_downloader('photos.getAlbums','owner_id='+his_id)['response']]
albums

[235698536,
 235422333,
 234565119,
 231698970,
 220214180,
 219249629,
 215459702,
 176201807,
 174855244,
 112238015]

In [67]:
# Фото из конкретного альбома
Vk_downloader('photos.get','owner_id='+his_id+'&album_id=235698536')['response'][0]

{'aid': 235698536,
 'created': 1472484553,
 'height': 1707,
 'owner_id': 10592581,
 'pid': 431139714,
 'src': 'https://pp.userapi.com/c637430/v637430581/7695/53uLqS8_MoA.jpg',
 'src_big': 'https://pp.userapi.com/c637430/v637430581/7696/dqjz9S35T2s.jpg',
 'src_small': 'https://pp.userapi.com/c637430/v637430581/7694/fI0hCs3Jem0.jpg',
 'src_xbig': 'https://pp.userapi.com/c637430/v637430581/7697/xfAmteKcR0Q.jpg',
 'src_xxbig': 'https://pp.userapi.com/c637430/v637430581/7698/eKrueDx4T60.jpg',
 'src_xxxbig': 'https://pp.userapi.com/c637430/v637430581/7699/FsiCf_iTMFY.jpg',
 'text': '',
 'width': 2560}

In [80]:
# Вытаскиваем айдишники всех фоток из всех альбомов, а также время когда фотка была залита. 
# К некоторым альбомам нам отказано в доступе... К каким не очень понятно.

photo_ids = [ ]
for alb in albums:
    try:
        cur = Vk_downloader('photos.get','owner_id='+his_id+'&album_id='+str(alb))['response']
        ids = [[item['pid'],item['created']] for item in cur]
        photo_ids.extend(ids)
    except Exception:
        print(alb)

176201807
174855244
112238015


In [82]:
# Количество фоточек
len(photo_ids)

1487

Собираем всю информацию о лайках под фотками за последнее время.

In [91]:
Vk_downloader('likes.getList','type=photo&owner_id='+his_id+'&item_id='+str(photo_ids[0][0]))

{'response': {'count': 0, 'users': []}}

In [92]:
photo_likes = [ ]
for item in photo_ids:
    a = Vk_downloader('likes.getList','type=photo&owner_id='+his_id+'&item_id='+str(item[0]))
    wall_likes.append([item[1],a['response']['users']])
    
photo_likes[-3:]

[]

## 2. Кого он лайкает

Он может быть очень хитрым. Вполне возможно, что он скрыл из списка своих друзей всех тех (ну вы поняли), чтобы конкретно ТЫ, моя милая воображалка, не нашла их. Нам нужно собрать id всех друзей его друзей. На всякий случай соберём в дополнение id всех друзей друзей его друзей. Найдём у всех этих множеств пересечение. Это будет список подозреваемых. 

Код для ультра-хардкорной рекурсивной процедуры сбора друзей друзей друзей от друзей друзей друзей можно найти в [соответствующей статейке на хабре.](https://habrahabr.ru/post/243229/)

Отлично! Мы на шаг ближе к разоблачению этого изменщика. Пришло время собирать лайки. Нас интересует лайк вполне конкретного человека. ЕГО ЛАЙК. Для нас достаточным было бы просто взять и записать все его лайки. Остальные лайки нас не волнуют. 

Однако запишем и их тоже, чтобы в самом конце скрипта запилить парочку красивых визуализаций. Все его лайки будем помещать в отдельный словарь. 

Все предположительные лайки предположительным... (ну вы поняли) собраны. Пришло время безжалостного и объективного анализа данных.


## 3. Проверка пары гипотез

Пока у нас окончательно не разыгрался синдром поиска измены, проверим парочку интересных гипотез.

* Гипотеза 1: взаимные лайки с какой вероятностью на лайк в течение 30 минут следует ответный лайк? 
* Гипотеза 2: в какое время суток парни лайкают дам
* Гипотеза 3: в какое время суток дамы лайкают парней
* Гипотеза 4: в какие дни недели ставятся лайки
* Гипотеза 5: 



## 4.  Взаимные лайки

 